# Heart Disease Prediction - Modelos Simples

Notebook dedicado a modelos clássicos de machine learning para a competição Kaggle Playground Series S6E2.
Modelos simples servem como **baseline** e trazem **diversidade** para o ensemble final com os modelos boost.

---

## Modelos
- Regressão Linear (Ridge)
- Regressão Logística
- Árvore de Decisão
- Random Forest

## Métrica: ROC-AUC

# 1. ENVIRONMENT SETUP

-------

## 1.1 Instalação de dependências

In [ ]:
# Installation of all necessary dependencies
!pip install -q scikit-learn pandas numpy matplotlib seaborn

## 1.2 Imports

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import KFold, StratifiedKFold, train_test_split, cross_val_score
from sklearn.metrics import roc_auc_score, roc_curve, confusion_matrix, classification_report
from sklearn.base import clone
from sklearn.preprocessing import StandardScaler
import warnings
warnings.filterwarnings('ignore')

RANDOM_STATE = 42 #Random seed para repordutibilidade
np.random.seed(RANDOM_STATE)

## 1.3 Data Loading

In [ ]:
test = pd.read_csv("data/test.csv") #carega dados de test
train = pd.read_csv("data/train.csv") #carrega dados de treino

print(f"Train shape: {train.shape}")
print(f"Test shape: {test.shape}")
print(f"\nTotal records: {train.shape[0] + test.shape[0]:,}") #quantidade todais de dados

In [ ]:
train.head() #primeiras 5 linhas

In [ ]:
train.columns

| Feature (Coluna) | Descrição | Tipo de Dado | Detalhes dos Valores |
| :--- | :--- | :--- | :--- |
| **Age** | Idade do paciente (em anos) | Numérico/Inteiro | Idade em anos. |
| **Sex** | Gênero do paciente | Categórico/Binário | 1 = Masculino<br>0 = Feminino |
| **Chest pain type** | Tipo de dor no peito | Categórico | 1 = Angina típica<br>2 = Angina atípica<br>3 = Dor não anginosa<br>4 = Assintomática |
| **BP** | Pressão arterial em repouso (mm Hg) | Numérico/Inteiro | Valor da pressão arterial em mm Hg. |
| **Cholesterol** | Nível de colesterol sérico (mg/dL) | Numérico/Inteiro | Valor do colesterol em mg/dL. |
| **FBS over 120** | Glicemia em jejum > 120 mg/dL | Categórico/Binário | 1 = Verdadeiro<br>0 = Falso |
| **EKG results** | Resultados de eletrocardiograma em repouso | Categórico | 0 = Normal<br>1 = Anormalidade da onda ST-T<br>2 = Hipertrofia ventricular esquerda |
| **Max HR** | Frequência cardíaca máxima alcançada | Numérico/Inteiro | Batimentos máximos alcançados. |
| **Exercise angina** | Angina induzida por exercício | Categórico/Binário | 1 = Sim<br>0 = Não |
| **ST depression** | Depressão do ST induzida por exercício em relação ao repouso | Numérico/Decimal | Valor da depressão do segmento ST. |
| **Slope of ST** | Inclinação do segmento ST do pico de exercício | Categórico | Descreve a inclinação do pico do exercício ST. |
| **Number of vessels fluro** | Número de vasos principais (0-3) coloridos por fluoroscopia | Numérico/Inteiro | Quantidade de vasos (0 a 3). |
| **Thallium** | Resultado do teste de estresse com Tálio (indicador médico categórico) | Categórico | Indicador médico categórico. |
| **Heart Disease** | Variável alvo | Categórico (Alvo) | Presence = Doença cardíaca detectada<br>Absence = Sem doença cardíaca |

In [ ]:
train.info()

In [ ]:
print(test.isnull().sum())

- Vemos a ausência de dados nulos, não necessitando de tratamento nessa etapa.

In [ ]:
train.describe() #estatisticas básicas

- Separar os tipos de features e também o target(variavel alvo)

In [ ]:
target = "Heart Disease"
id_col = "id"

numeric_features = [
    "Age",
    "BP",
    "Cholesterol",
    "Max HR",
    "ST depression"
]

categorical_features = [
    "Sex",
    "Chest pain type",
    "FBS over 120",
    "EKG results",
    "Exercise angina",
    "Slope of ST",
    "Number of vessels fluro",
    "Thallium"
]

print(f"Target variable: '{target}'")
print(f"ID column: '{id_col}'")
print(f"\nNumeric features ({len(numeric_features)}): {numeric_features}")
print(f"\nCategorical features ({len(categorical_features)}): {categorical_features}")
print(f"\nTotal features: {len(numeric_features) + len(categorical_features)}")

In [ ]:
x_train = train.drop([id_col, target], axis=1) #x sendo os dados de treino sem o id (nao ajudara em nada no modelo e o target)
y_train = train[target] #y sendo o target
X_test = test.drop([id_col], axis=1) #x_teste sendo os dados sem ID, dados de teste não tem o targe (A nossa missão é descobrir)

print(x_train.shape, y_train.shape, X_test.shape)

# 2. EDA

------------------

## 2.1 Análise da variável Alvo

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(14, 5))
target_counts = y_train.value_counts() #contagem de dados target, devolverá a contagem de pessoas com doença e sem doença
target_pct = y_train.value_counts(normalize=True) * 100 #normaliza a contagem onde a soma total é 100%
print(target_pct)

axes[0].bar(target_counts.index, target_counts.values, color=['#2ecc71', '#e74c3c'], edgecolor='black') #Grafico de barra com os valores absoluotos de contagem
axes[0].set_title('Heart Disease Distribution (Counts)', fontsize=14, fontweight='bold')
axes[0].set_xlabel('Heart Disease')
axes[0].set_ylabel('Count')
axes[0].set_xticks([0, 1])
axes[0].set_xticklabels(['No Disease (0)', 'Disease (1)'])
for i, v in enumerate(target_counts.values): #para aparecer o numero nas barras
    axes[0].text(i, v + 1000, f'{v:,}', ha='center', fontweight='bold')

axes[1].pie(target_counts.values, labels=['No Disease (0)', 'Disease (1)'], #grafico de pizza com os valores normalizados
            autopct='%1.1f%%', startangle=90, colors=['#2ecc71', '#e74c3c'])
axes[1].set_title('Heart Disease Distribution (%)', fontsize=14, fontweight='bold')

plt.tight_layout()
plt.show()

Essa proporção é ideal para o treinamento de modelos preditivos, pois a pequena diferença entre as classes minimiza o risco de viés e garante representatividade para ambos os grupos.

## 2.2. Correlação entre features numéricas e target

In [ ]:
numeric_data = x_train[numeric_features].copy()
if y_train.dtype == 'object': #convert para dados numericas doneças ou sem doenças
    y_numeric = y_train.map({'Absence': 0, 'Presence': 1}).values #convert 0 e 1
    print(f"Unique values: {y_train.unique()} -> {np.unique(y_numeric)}")
else:
    y_numeric = y_train


numeric_data['Heart Disease'] = y_numeric
correlation_matrix = numeric_data.corr() #matriz de correlação
target_corr = correlation_matrix['Heart Disease'].drop('Heart Disease').sort_values(ascending=False)
fig, axes = plt.subplots(1, 2, figsize=(16, 6))
sns.heatmap(correlation_matrix, annot=True, fmt='.3f', cmap='coolwarm', center=0, 
            ax=axes[0], linewidths=0.5, cbar_kws={'label': 'Correlation'})
axes[0].set_title('Correlation Matrix (Numeric Features + Target)', fontsize=14, fontweight='bold')


target_corr.plot(kind='barh', ax=axes[1], color=['green' if x > 0 else 'red' for x in target_corr]) #para saber quas influenciam em doenças
axes[1].set_title('Feature Correlation with Heart Disease', fontsize=14, fontweight='bold')
axes[1].set_xlabel('Correlation Coefficient')
axes[1].axvline(0, color='black', linewidth=0.8)


plt.tight_layout()
plt.show()

## 2.3 Categorical Features vs Target Analysis

In [ ]:
n_categorical = len(categorical_features) #quantidade de variaveis categorias
n_cols = 3
n_rows = (n_categorical + n_cols - 1) // n_cols #quantidade de linha para plots , 3 linhas de graficos
fig, axes = plt.subplots(n_rows, n_cols, figsize=(18, n_rows * 4)) #subplot 3x3
axes = axes.flatten()

for idx, feature in enumerate(categorical_features):
    ax = axes[idx]
    cross_tab = pd.crosstab(x_train[feature], y_train, normalize='index') * 100 #Cruza os dados para contar quantos pacientes existem em cada combinação
    #Transforma os números brutos em proporções dentro de cada categoria. 
    cross_tab.plot(kind='bar', stacked=True, ax=ax, color=['#2ecc71', '#e74c3c'],  #plot das barras
                   edgecolor='black', legend=False)
    ax.set_title(f'{feature} vs Heart Disease', fontweight='bold')#titulo grafico
    ax.set_xlabel(feature)
    ax.set_ylabel('Percentage (%)')
    ax.set_xticklabels(ax.get_xticklabels(), rotation=45, ha='right')
    ax.grid(alpha=0.3, axis='y')

for idx in range(n_categorical, len(axes)):
    axes[idx].axis('off')

handles = [plt.Rectangle((0,0),1,1, color='#2ecc71'), plt.Rectangle((0,0),1,1, color='#e74c3c')]#cores legenda
labels = ['No Disease (0)', 'Disease (1)'] #labels leganda
fig.legend(handles, labels, loc='upper right', fontsize=12)

plt.tight_layout()
plt.show()

### Análise das Variáveis Categóricas (Incidência de Doença)

* **Sex:** Apresenta uma disparidade significativa; o grupo masculino (1) demonstra uma taxa de incidência de doença consideravelmente superior ao grupo feminino (0), sendo um forte divisor demográfico no modelo.
* **Chest pain type:** O tipo 4 (Assintomática) é o indicador mais crítico, apresentando as maiores taxas de presença da doença, enquanto o tipo 2 (Angina atípica) apresenta a menor incidência proporcional.
* **FBS over 120:** Embora indique risco metabólico, a taxa de doença entre quem tem glicemia alta (>120) é muito similar à de quem tem glicemia normal, sugerindo ser o preditor categórico de menor impacto individual.
* **EKG results:** Pacientes com resultados tipo 2 (Hipertrofia ventricular esquerda) mostram uma probabilidade de doença cardíaca visivelmente maior em comparação aos que apresentam resultados normais (0).
* **Exercise angina:** Um dos preditores mais binários; a presença de angina induzida por exercício (1) eleva drasticamente a taxa de doença cardíaca positiva em comparação a quem não manifesta o sintoma (0).
* **Slope of ST:** A inclinação descendente ou plana no teste de esforço está fortemente correlacionada à presença da doença, enquanto a inclinação ascendente é um forte indicador de ausência (saúde cardíaca).
* **Thallium:** Variável de alta especificidade; o resultado "Reversível" apresenta uma taxa de doença muito superior ao resultado "Normal", tornando-se um dos filtros mais precisos para o diagnóstico final.

# 3. DATA PREPROCESSING

---------------

- Vimos acima que somente o Target(y) necessita de transformação para numero, pois ele tem presence e absence e não valores númericos como 0 e 1

## 3.1 Categorical Variables Transformation

In [ ]:
y_train_transformed = y_train.map({'Absence': 0, 'Presence': 1}).values #absence tera valor 0 e presença valor
print(f"\nTarget shape: {y_train_transformed.shape}")
print(f"Unique values: {np.unique(y_train_transformed)}")
print(f"Distribution: {np.bincount(y_train_transformed)}")

- vemos agora que so tem valores 0 e 1, ideal para o modelo que queremos trabalhar.

## 3.2 Feature Engineering

In [ ]:
from sklearn.linear_model import LogisticRegression

scaler_lr = StandardScaler()# Treinar regressão logística simples
X_scaled = scaler_lr.fit_transform(x_train[numeric_features]) #normaliza dados numerricos
lr = LogisticRegression(random_state=42)
lr.fit(X_scaled, y_train_transformed) #treina modelo

# Extrair coeficientes para usar na criaçao e features
coefficients = dict(zip(numeric_features, lr.coef_[0]))
print("Coeficientes aprendidos:")
for feat, coef in coefficients.items():
    print(f"  {feat}: {coef:.4f}")

In [ ]:
def create_features(df):
    df = df.copy()
    eps = 1e-6
    
    # 1. Polynomial features (Square)
    df["Age_sq"] = df["Age"] ** 2
    df["Cholesterol_sq"] = df["Cholesterol"] ** 2
    df["Max HR_sq"] = df["Max HR"] ** 2
    df["ST depression_sq"] = df["ST depression"] ** 2
    
    # 2. Mathematical transformations (Log/Sqrt)
    df["log_age"] = np.log1p(df["Age"])
    df["log_cholesterol"] = np.log1p(df["Cholesterol"])
    df["log_st_depression"] = np.log1p(df["ST depression"])
    df["sqrt_age"] = np.sqrt(df["Age"])
    df["sqrt_cholesterol"] = np.sqrt(df["Cholesterol"])
    df["sqrt_maxhr"] = np.sqrt(df["Max HR"])
    
    # 3. Interactions (Multiplications)
    df["age_x_st_depression"] = df["Age"] * df["ST depression"]
    df["maxhr_x_st_depression"] = df["Max HR"] * df["ST depression"]
    df["age_x_maxhr"] = df["Age"] * df["Max HR"]
    df["age_x_cholesterol"] = df["Age"] * df["Cholesterol"]
    df["cholesterol_x_st_depression"] = df["Cholesterol"] * df["ST depression"]

    # 4. Ratios (Divisions)
    df["cholesterol_per_age"] = df["Cholesterol"] / (df["Age"] + eps)
    df["maxhr_per_age"] = df["Max HR"] / (df["Age"] + eps)
 
    # 5. Differences (Deviation from mean)
    df["age_diff_mean"] = df["Age"] - df["Age"].mean()
    df["cholesterol_diff_mean"] = df["Cholesterol"] - df["Cholesterol"].mean()
    df["maxhr_diff_mean"] = df["Max HR"] - df["Max HR"].mean()
    
    # 6. Binning (Discretization)
    df["age_bin"] = pd.cut(df["Age"], bins=[-1, 40, 50, 60, 100], labels=False)
    df["cholesterol_bin"] = pd.cut(df["Cholesterol"], bins=[-1, 200, 240, 280, 600], labels=False)
    df["maxhr_bin"] = pd.cut(df["Max HR"], bins=[-1, 100, 130, 160, 250], labels=False)
    df["st_bin"] = pd.cut(df["ST depression"], bins=[-1, 0.5, 1.5, 3.0, 10], labels=False)
    
    # 7. Boolean Flags (Risk factors)
    df["elderly"] = (df["Age"] >= 60).astype(int)
    df["high_cholesterol"] = (df["Cholesterol"] >= 240).astype(int)
    df["high_st_depression"] = (df["ST depression"] > 2.0).astype(int)
    
    # 8. Risk Scores
    df["learned_risk_score"] = (
        df["Age"] * 0.4244 +
        df["BP"] * (-0.0044) +
        df["Cholesterol"] * 0.1441 +
        df["Max HR"] * (-1.0234) +
        df["ST depression"] * 0.9945
    )
    
    df["risk_factors_count"] = (
        df["elderly"] + 
        df["high_cholesterol"] + 
        df["high_st_depression"]
    )
    
    return df

In [ ]:
x_train_features = create_features(x_train)
X_test_features  = create_features(X_test)
print(f"Features após create_features: {x_train_features.shape[1]}")

# --- 9. Frequency Encoding ---
all_data = pd.concat([x_train_features, X_test_features])
all_features = numeric_features + categorical_features
for col in all_features:
    freq = all_data[col].value_counts(normalize=True)
    x_train_features[col + "_freq"] = x_train_features[col].map(freq)
    X_test_features[col + "_freq"] = X_test_features[col].map(freq)
print(f"Features após Frequency Encoding: {x_train_features.shape[1]}")

## 3.3 Feature Quality Analysis

In [ ]:
# Correlation with target
df_corr = x_train_features.copy()
df_corr['target'] = y_train_transformed
corr_target = df_corr.corr(numeric_only=True)['target'].drop('target') #calcula correlação com o target
corr_sorted = corr_target.abs().sort_values(ascending=False)

#Visualization Features
height = 12
plt.figure(figsize=(14, height))

top_corr = corr_target.loc[corr_sorted.head(42).index]
colors = ['green' if x > 0 else 'red' for x in top_corr.values]

bars = plt.barh(range(len(top_corr)), top_corr.values, color=colors)
plt.yticks(range(len(top_corr)), top_corr.index)
plt.xlabel('Correlation Coefficient', fontsize=12)
plt.title('Top Features - Correlation with Heart Disease', fontsize=14, fontweight='bold')
plt.axvline(0, color='black', linewidth=0.8)
plt.grid(alpha=0.3, axis='x')

for i, (feat, val) in enumerate(zip(top_corr.index, top_corr.values)):
    plt.text(val + 0.01 if val > 0 else val - 0.01, i, f'{val:.3f}', 
             va='center', ha='left' if val > 0 else 'right', fontsize=9)
plt.tight_layout()
plt.show()

## 3.4 Model Evaluation Function (Cross-Validation)

In [ ]:
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score
from sklearn.base import clone
import numpy as np

def evaluate_model_cv(model, X, y, X_test=None, cv=5):
    """
    Avalia modelo com cross-validation e retorna:
    - roc_scores: array com ROC-AUC de cada fold
    - oof_preds: predições out-of-fold no treino
    - test_preds: predições no teste (média dos folds) - só se X_test for passado
    """
    skf = StratifiedKFold(n_splits=cv, shuffle=True, random_state=42)
    roc_auc_scores = []
    oof_preds = np.zeros(len(y))
    
    if X_test is not None:
        test_preds_folds = np.zeros((len(X_test), cv))
    
    for fold, (train_idx, val_idx) in enumerate(skf.split(X, y)):
        if hasattr(X, 'iloc'):
            X_train_fold = X.iloc[train_idx]
            X_val_fold = X.iloc[val_idx]
        else:
            X_train_fold = X[train_idx]
            X_val_fold = X[val_idx]
        
        y_train_fold = y[train_idx]
        y_val_fold = y[val_idx]
        model_fold = clone(model)
        
        model_fold.fit(X_train_fold, y_train_fold)
        oof_preds[val_idx] = model_fold.predict_proba(X_val_fold)[:, 1]
        
        if X_test is not None:
            test_preds_folds[:, fold] = model_fold.predict_proba(X_test)[:, 1]
        
        roc_auc = roc_auc_score(y_val_fold, oof_preds[val_idx])
        roc_auc_scores.append(roc_auc)
    
    if X_test is not None:
        test_preds = test_preds_folds.mean(axis=1)
        return np.array(roc_auc_scores), oof_preds, test_preds
    
    return np.array(roc_auc_scores), oof_preds

## 3.5 Normalização (StandardScaler)

Regressão Linear e Logística são sensíveis à escala dos dados. Normalizamos para que todas as features contribuam igualmente.

In [ ]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(x_train_features)
X_test_scaled = scaler.transform(X_test_features)

print(f"X_train_scaled shape: {X_train_scaled.shape}")
print(f"X_test_scaled shape: {X_test_scaled.shape}")

# 4. Models

## 4.1 Regressão Linear (Ridge)

----------

Usa regressão linear com regularização L2 (Ridge) para prever probabilidades. Como é um problema de classificação, fazemos clip das predições entre 0 e 1.

In [ ]:
from sklearn.linear_model import Ridge

# Regressão Linear não tem predict_proba, então fazemos CV manual
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
roc_auc_scores_ridge = []
oof_ridge = np.zeros(len(y_train_transformed))
test_preds_ridge = np.zeros((len(X_test_scaled), 5))

for fold, (train_idx, val_idx) in enumerate(skf.split(X_train_scaled, y_train_transformed)):
    X_tr = X_train_scaled[train_idx]
    X_val = X_train_scaled[val_idx]
    y_tr = y_train_transformed[train_idx]
    y_val = y_train_transformed[val_idx]
    
    ridge = Ridge(alpha=1.0, random_state=42)
    ridge.fit(X_tr, y_tr)
    
    # Clip entre 0 e 1 para simular probabilidades
    oof_ridge[val_idx] = np.clip(ridge.predict(X_val), 0, 1)
    test_preds_ridge[:, fold] = np.clip(ridge.predict(X_test_scaled), 0, 1)
    
    roc_auc = roc_auc_score(y_val, oof_ridge[val_idx])
    roc_auc_scores_ridge.append(roc_auc)

y_test_proba_ridge = test_preds_ridge.mean(axis=1)

for fold, score in enumerate(roc_auc_scores_ridge, 1):
    print(f"  Fold {fold}: {score:.4f}")
print(f"\nMean ROC-AUC: {np.mean(roc_auc_scores_ridge):.4f}")

In [ ]:
submission_ridge = pd.DataFrame({
    'id': test['id'],
    'Heart Disease': y_test_proba_ridge
})

submission_ridge.to_csv('submission_Ridge.csv', index=False)
print("Ridge submission salvo: submission_Ridge.csv")

## 4.2 Regressão Logística

----------

Modelo linear clássico para classificação binária. Usa dados normalizados para melhor convergência.

In [ ]:
from sklearn.linear_model import LogisticRegression

logreg_model = LogisticRegression(
    C=1.0,
    max_iter=1000,
    random_state=42,
    n_jobs=-1
)

# Usa dados normalizados
roc_scores_lr, oof_logreg, y_test_proba_logreg = evaluate_model_cv(
    logreg_model, X_train_scaled, y_train_transformed, X_test_scaled, cv=5
)

for fold, score in enumerate(roc_scores_lr, 1):
    print(f"  Fold {fold}: {score:.4f}")
print(f"\nMean ROC-AUC: {roc_scores_lr.mean():.4f}")

In [ ]:
submission_logreg = pd.DataFrame({
    'id': test['id'],
    'Heart Disease': y_test_proba_logreg
})

submission_logreg.to_csv('submission_LogReg.csv', index=False)
print("Logistic Regression submission salvo: submission_LogReg.csv")

## 4.3 Árvore de Decisão

----------

Modelo baseado em regras de decisão. Não precisa de normalização. Serve como baseline para comparar com Random Forest.

In [ ]:
from sklearn.tree import DecisionTreeClassifier

dt_model = DecisionTreeClassifier(
    max_depth=10,
    min_samples_split=20,
    min_samples_leaf=10,
    random_state=42
)

# Árvore não precisa de normalização, usa features originais
roc_scores_dt, oof_dt, y_test_proba_dt = evaluate_model_cv(
    dt_model, x_train_features, y_train_transformed, X_test_features, cv=5
)

for fold, score in enumerate(roc_scores_dt, 1):
    print(f"  Fold {fold}: {score:.4f}")
print(f"\nMean ROC-AUC: {roc_scores_dt.mean():.4f}")

In [ ]:
submission_dt = pd.DataFrame({
    'id': test['id'],
    'Heart Disease': y_test_proba_dt
})

submission_dt.to_csv('submission_DecisionTree.csv', index=False)
print("Decision Tree submission salvo: submission_DecisionTree.csv")

## 4.4 Random Forest

----------

Ensemble de árvores de decisão. Combina múltiplas árvores para reduzir variância e melhorar generalização.

In [ ]:
from sklearn.ensemble import RandomForestClassifier

rf_model = RandomForestClassifier(
    n_estimators=200,
    max_depth=10,
    random_state=42,
    n_jobs=-1
)

roc_scores_rf, oof_rf, y_test_proba_rf = evaluate_model_cv(
    rf_model, x_train_features, y_train_transformed, X_test_features, cv=5
)

for fold, score in enumerate(roc_scores_rf, 1):
    print(f"  Fold {fold}: {score:.4f}")
print(f"\nMean ROC-AUC: {roc_scores_rf.mean():.4f}")

In [ ]:
submission_rf = pd.DataFrame({
    'id': test['id'],
    'Heart Disease': y_test_proba_rf
})

submission_rf.to_csv('submission_RF.csv', index=False)
print("Random Forest submission salvo: submission_RF.csv")

## 4.5 Comparação dos Modelos

In [ ]:
# Resumo de todos os modelos
models_summary = {
    'Ridge': np.mean(roc_auc_scores_ridge),
    'Logistic Regression': roc_scores_lr.mean(),
    'Decision Tree': roc_scores_dt.mean(),
    'Random Forest': roc_scores_rf.mean(),
}

print("=" * 45)
print(f"{'Modelo':<25} {'ROC-AUC':>10}")
print("=" * 45)
for model, score in sorted(models_summary.items(), key=lambda x: x[1], reverse=True):
    print(f"{model:<25} {score:>10.4f}")
print("=" * 45)

# Gráfico comparativo
fig, ax = plt.subplots(figsize=(10, 5))
names = list(models_summary.keys())
scores = list(models_summary.values())
colors = ['#3498db', '#2ecc71', '#e67e22', '#e74c3c']
bars = ax.bar(names, scores, color=colors, edgecolor='black')
ax.set_ylabel('ROC-AUC')
ax.set_title('Comparação dos Modelos Simples', fontweight='bold')
ax.set_ylim(min(scores) - 0.02, max(scores) + 0.01)
for bar, score in zip(bars, scores):
    ax.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.002,
            f'{score:.4f}', ha='center', fontweight='bold')
ax.grid(alpha=0.3, axis='y')
plt.tight_layout()
plt.show()

## 4.6 Ensemble Simples

Média simples das probabilidades dos 4 modelos.

In [ ]:
# Ensemble Simples - média das probabilidades dos 4 modelos
y_ensemble_simple = (y_test_proba_ridge + y_test_proba_logreg + y_test_proba_dt + y_test_proba_rf) / 4

submission_ensemble_simple = pd.DataFrame({
    'id': test['id'],
    'Heart Disease': y_ensemble_simple
})

submission_ensemble_simple.to_csv('submission_ensemble_simples.csv', index=False)
print("Ensemble Simples salvo: submission_ensemble_simples.csv")

## 4.7 Ensemble Ponderado

Média ponderada das probabilidades, com pesos proporcionais ao score OOF de cada modelo.

In [ ]:
# Scores calculados automaticamente a partir do OOF de cada modelo
scores = {
    'ridge': roc_auc_score(y_train_transformed, oof_ridge),
    'logreg': roc_auc_score(y_train_transformed, oof_logreg),
    'dt': roc_auc_score(y_train_transformed, oof_dt),
    'rf': roc_auc_score(y_train_transformed, oof_rf)
}

# Pesos normalizados proporcionais ao score
total = sum(scores.values())
w_ridge = scores['ridge'] / total
w_logreg = scores['logreg'] / total
w_dt = scores['dt'] / total
w_rf = scores['rf'] / total

print(f"Scores OOF:")
print(f"  Ridge:              {scores['ridge']:.6f} (peso: {w_ridge:.4f})")
print(f"  Logistic Regression:{scores['logreg']:.6f} (peso: {w_logreg:.4f})")
print(f"  Decision Tree:      {scores['dt']:.6f} (peso: {w_dt:.4f})")
print(f"  Random Forest:      {scores['rf']:.6f} (peso: {w_rf:.4f})")

# Ensemble Ponderado
y_ensemble_weighted = (w_ridge * y_test_proba_ridge) + (w_logreg * y_test_proba_logreg) + \
                      (w_dt * y_test_proba_dt) + (w_rf * y_test_proba_rf)

submission_ensemble_weighted = pd.DataFrame({
    'id': test['id'],
    'Heart Disease': y_ensemble_weighted
})

submission_ensemble_weighted.to_csv('submission_ensemble_simples_weighted.csv', index=False)
print("\nEnsemble ponderado salvo: submission_ensemble_simples_weighted.csv")